In [24]:
import random
from typing import List, Tuple


class Object:
    def __init__(
        self,
        name: str,
        type: str,
    ) -> None:
        """Entity, e.g., human, object, room.

        Args
        ----
        name: e.g., Tae, laptop, bed
        type: e.g., static, independent, dependent
        """
        self.name = name
        self.type = type

    def __repr__(self) -> str:
        return f"Object({self.name}, {self.type})"

    def move(self) -> None:
        """Move object to another room."""
        pass


class StaticObject(Object):
    def __init__(self, name: str, location: str) -> None:
        super().__init__(name, "static")
        self.location = location

    def __repr__(self) -> str:
        return f"StaticObject({self.name}, {self.location})"

    def move(self) -> None:
        """Static object cannot move."""
        raise ValueError("Static object cannot move.")


class IndepdentObject(Object):
    def __init__(self, name: str, probs: dict) -> None:
        super().__init__(name, "independent")
        self.probs = probs
        self.move()

    def __repr__(self) -> str:
        return f"IndependentObject({self.name}, {self.location})"

    def move(self) -> None:
        """Indendent object objects to another room."""
        self.location = random.choices(
            list(self.probs.keys()),
            weights=list(self.probs.values()),
            k=1,
        )[0]


class DependentObject(Object):
    def __init__(
        self, name: str, dependence: List[Tuple[IndepdentObject, float]]
    ) -> None:
        super().__init__(name, "dependent")
        self.dependence = dependence
        independent_object, prob = random.choice(self.dependence)
        self.attached = independent_object
        self.location = self.attached.location

    def attach(self, independent_objects: List[str]) -> None:
        """Attach dependent object to independent object."""
        possible = []
        for io in independent_objects:
            for dep in self.dependence:
                if dep[0].name == io:
                    possible.append(dep)

        independent_object, prob = random.choice(possible)

        if random.random() < prob:
            self.attached = independent_object
        else:
            self.attached = None

    def move(self) -> None:
        """Move together with independent object."""
        if self.attached is not None:
            self.location = self.attached.location

    def __repr__(self) -> str:
        return f"DependentObject({self.name}, {self.location}, {self.attached})"


class Agent(Object):
    def __init__(self, init_probs: dict) -> None:
        super().__init__("agent", "agent")
        self.location = random.choices(
            list(init_probs.keys()),
            weights=list(init_probs.values()),
            k=1,
        )[0]

    def __repr__(self) -> str:
        return f"Agent({self.name}, {self.location})"

    def move(self, location: str) -> None:
        """Agent can choose where to go."""
        self.location = location


class Rooms:
    def __init__(
        self,
        rooms: List[str],
        static_objects: List[StaticObject],
        independent_objects: List[IndepdentObject],
        dependent_objects: List[DependentObject],
        agent: Agent,
        seed: int,
    ) -> None:
        self.rooms = {room: [] for room in rooms}
        self.static_objects = static_objects
        self.independent_objects = independent_objects
        self.dependent_objects = dependent_objects
        self.agent = agent
        self.seed = seed

        random.seed(seed)

        for obj in self.static_objects:
            self.rooms[obj.location].append(obj)

        for obj in self.independent_objects:
            self.rooms[obj.location].append(obj)

        for obj in self.dependent_objects:
            self.rooms[obj.location].append(obj)

        self.rooms[agent.location].append(agent)

    def reset(self) -> Tuple[List]:
        """Agent observes a sub graph and a question."""
        self.time = 0
        return self.get_sub_graph(), self.get_question()

    def step(self, action: int) -> Tuple[List]:
        """Agent takes an action to move to one of the rooms."""
        for room, val in self.rooms.items():
            local_independent_objects = []
            local_dependent_objects = []

            for val_ in val:
                if isinstance(val_, IndepdentObject):
                    local_independent_objects.append(val_)
                elif isinstance(val_, DependentObject):
                    local_dependent_objects.append(val_)

            if local_independent_objects and local_dependent_objects:
                for dep in local_dependent_objects:
                    dep.attach([ind.name for ind in local_independent_objects])

            for ind in local_independent_objects:
                ind.move()

            for dep in local_dependent_objects:
                dep.move()

        self.agent.move(list(self.rooms.keys())[action])

        for obj in self.static_objects:
            self.rooms[obj.location].append(obj)

        for obj in self.independent_objects:
            self.rooms[obj.location].append(obj)

        for obj in self.dependent_objects:
            self.rooms[obj.location].append(obj)

        self.rooms[self.agent.location].append(self.agent)

        self.hidden_state = self.get_hidden_state()

        self.time = +1

        return self.get_sub_graph(), self.get_question()

    def get_hidden_state(self) -> List:
        self.hidden_state = []

        for room, val in self.rooms.items():
            for val_ in val:
                self.hidden_state.append([val_.name, "AtLocation", room, self.time])

        return self.hidden_state

    def get_sub_graph(self) -> List:
        self.sub_graph = []

        agent_room = None
        for key, val in self.rooms.items():
            for val_ in val:
                if val_.name == "agent":
                    agent_room = key
                if agent_room is not None:
                    break
            if agent_room is not None:
                break

        for obj in self.rooms[agent_room]:
            self.sub_graph.append([obj.name, "AtLocation", agent_room, self.time])

        return self.sub_graph

    def get_question(self) -> List:
        self.hidden_state = self.get_hidden_state()

        while True:
            self.question = random.choice(self.hidden_state)
            if self.question[0] != "agent":
                break

        return self.question

    def __repr__(self) -> str:
        return f"Rooms({self.rooms})"


In [25]:
# static objects
bed = StaticObject("bed", "bedroom")
desk = StaticObject("desk", "officeroom")
table = StaticObject("table", "livingroom")

# independent objects
tae = IndepdentObject("tae", {"officeroom": 0.5, "livingroom": 0.5})
michael = IndepdentObject("michael", {"bedroom": 0.5, "livingroom": 0.5})
vincent = IndepdentObject("vincent", {"bedroom": 0.5, "officeroom": 0.5})

# dependent objects
laptop = DependentObject("laptop", [(tae, 0.7), (michael, 0.1), (vincent, 0.3)])
phone = DependentObject("phone", [(tae, 0.3), (michael, 0.1), (vincent, 0.7)])
headset = DependentObject("headset", [(tae, 0.5), (michael, 0.5), (vincent, 0.5)])
keyboard = DependentObject("keyboard", [(tae, 0.9), (michael, 0.7), (vincent, 0.5)])

# agent
agent = Agent({"bedroom": 0.333, "officeroom": 0.333, "livingroom": 0.333})

# rooms
rooms = Rooms(
    ["bedroom", "officeroom", "livingroom"],
    [bed, desk, table],
    [tae, michael, vincent],
    [laptop, phone, headset, keyboard],
    agent,
    42
)

In [31]:
rooms.get_hidden_state()

[['bed', 'AtLocation', 'bedroom', 0],
 ['vincent', 'AtLocation', 'bedroom', 0],
 ['phone', 'AtLocation', 'bedroom', 0],
 ['keyboard', 'AtLocation', 'bedroom', 0],
 ['bed', 'AtLocation', 'bedroom', 0],
 ['phone', 'AtLocation', 'bedroom', 0],
 ['agent', 'AtLocation', 'bedroom', 0],
 ['desk', 'AtLocation', 'officeroom', 0],
 ['tae', 'AtLocation', 'officeroom', 0],
 ['laptop', 'AtLocation', 'officeroom', 0],
 ['headset', 'AtLocation', 'officeroom', 0],
 ['desk', 'AtLocation', 'officeroom', 0],
 ['tae', 'AtLocation', 'officeroom', 0],
 ['vincent', 'AtLocation', 'officeroom', 0],
 ['laptop', 'AtLocation', 'officeroom', 0],
 ['headset', 'AtLocation', 'officeroom', 0],
 ['keyboard', 'AtLocation', 'officeroom', 0],
 ['table', 'AtLocation', 'livingroom', 0],
 ['michael', 'AtLocation', 'livingroom', 0],
 ['agent', 'AtLocation', 'livingroom', 0],
 ['table', 'AtLocation', 'livingroom', 0],
 ['michael', 'AtLocation', 'livingroom', 0]]

In [28]:
rooms.step(0)

([['bed', 'AtLocation', 'bedroom', 1],
  ['vincent', 'AtLocation', 'bedroom', 1],
  ['phone', 'AtLocation', 'bedroom', 1],
  ['keyboard', 'AtLocation', 'bedroom', 1],
  ['bed', 'AtLocation', 'bedroom', 1],
  ['phone', 'AtLocation', 'bedroom', 1],
  ['agent', 'AtLocation', 'bedroom', 1]],
 ['bed', 'AtLocation', 'bedroom', 1])

In [ ]:
# static objects
bed = StaticObject("bed", "bedroom")
desk = StaticObject("desk", "officeroom")
table = StaticObject("table", "livingroom")

# independent objects
tae = IndepdentObject("tae", {"officeroom": 0.5, "livingroom": 0.5})
michael = IndepdentObject("michael", {"bedroom": 0.5, "livingroom": 0.5})
vincent = IndepdentObject("vincent", {"bedroom": 0.5, "officeroom": 0.5})

# dependent objects
laptop = DependentObject("laptop", [(tae, 0.7), (michael, 0.1), (vincent, 0.3)])
phone = DependentObject("phone", [(tae, 0.3), (michael, 0.1), (vincent, 0.7)])

# agent
agent = Agent({"bedroom": 0.333, "officeroom": 0.333, "livingroom": 0.333})

# rooms
rooms = Rooms(
    ["bedroom", "officeroom", "livingroom"],
    [bed, desk, table],
    [tae, michael, vincent],
    [laptop, phone],
    agent,
)


In [263]:
rooms

Rooms({'bedroom': [StaticObject(bed, bedroom), IndependentObject(vincent, bedroom), DependentObject(phone, bedroom, IndependentObject(vincent, bedroom)), StaticObject(bed, bedroom), IndependentObject(vincent, bedroom), DependentObject(phone, bedroom, IndependentObject(vincent, bedroom)), Agent(agent, bedroom), StaticObject(bed, bedroom), IndependentObject(michael, bedroom), IndependentObject(vincent, bedroom), DependentObject(phone, bedroom, IndependentObject(vincent, bedroom)), StaticObject(bed, bedroom), IndependentObject(michael, bedroom), DependentObject(phone, bedroom, IndependentObject(vincent, bedroom)), StaticObject(bed, bedroom), IndependentObject(michael, bedroom), IndependentObject(vincent, bedroom), DependentObject(phone, bedroom, IndependentObject(vincent, bedroom)), Agent(agent, bedroom)], 'officeroom': [StaticObject(desk, officeroom), Agent(agent, bedroom), StaticObject(desk, officeroom), StaticObject(desk, officeroom), Agent(agent, bedroom), StaticObject(desk, officeroo

In [254]:
rooms.get_hidden_state()

[['bed', 'AtLocation', 'bedroom'], ['vincent', 'AtLocation', 'bedroom'], ['phone', 'AtLocation', 'bedroom'], ['desk', 'AtLocation', 'officeroom'], ['agent', 'AtLocation', 'officeroom'], ['table', 'AtLocation', 'livingroom'], ['tae', 'AtLocation', 'livingroom'], ['michael', 'AtLocation', 'livingroom'], ['laptop', 'AtLocation', 'livingroom']]

In [255]:
rooms.get_sub_graph()

[['desk', 'AtLocation', 'officeroom'], ['agent', 'AtLocation', 'officeroom']]

In [256]:
rooms.reset()

([['desk', 'AtLocation', 'officeroom'], ['agent', 'AtLocation', 'officeroom']], ['bed', 'AtLocation', 'bedroom'])

In [262]:
rooms.step(0)

([['bed', 'AtLocation', 'bedroom'], ['vincent', 'AtLocation', 'bedroom'], ['phone', 'AtLocation', 'bedroom'], ['bed', 'AtLocation', 'bedroom'], ['vincent', 'AtLocation', 'bedroom'], ['phone', 'AtLocation', 'bedroom'], ['agent', 'AtLocation', 'bedroom'], ['bed', 'AtLocation', 'bedroom'], ['michael', 'AtLocation', 'bedroom'], ['vincent', 'AtLocation', 'bedroom'], ['phone', 'AtLocation', 'bedroom'], ['bed', 'AtLocation', 'bedroom'], ['michael', 'AtLocation', 'bedroom'], ['phone', 'AtLocation', 'bedroom'], ['bed', 'AtLocation', 'bedroom'], ['michael', 'AtLocation', 'bedroom'], ['vincent', 'AtLocation', 'bedroom'], ['phone', 'AtLocation', 'bedroom'], ['agent', 'AtLocation', 'bedroom']], ['phone', 'AtLocation', 'bedroom'])

In [240]:
rooms.get_question()

['michael', 'AtLocation', 'bedroom']

In [192]:
rooms.dependent_objects[0]

DependentObject(laptop, officeroom, IndependentObject(tae, officeroom))

In [150]:
laptop = DependentObject("laptop", [(tae, 0.7), (michael, 0.1), (vincent, 0.3)])
print(laptop.attached)

IndependentObject(michael, independent, livingroom)


In [165]:
laptop.attach(["tae"])
print(laptop.attached)

IndependentObject(tae, independent, officeroom)


In [131]:
laptop.attach()

TypeError: list indices must be integers or slices, not str

In [91]:
random.random()

0.5958237632388955

In [87]:
random.choice(laptop.dependence)

(IndependentObject(vincent, independent, bedroom), 0.3)

In [80]:
laptop

DependentObject(laptop, dependent, [(IndependentObject(tae, independent, officeroom), 0.7), (IndependentObject(michael, independent, livingroom), 0.1), (IndependentObject(vincent, independent, bedroom), 0.3)])

In [82]:
laptop.dependence

[(IndependentObject(tae, independent, officeroom), 0.7),
 (IndependentObject(michael, independent, livingroom), 0.1),
 (IndependentObject(vincent, independent, bedroom), 0.3)]

In [68]:
agent.move("officeroom")
agent.location

'officeroom'

In [38]:
tae.move()
tae

IndependentObject(Tae, independent, officeroom)

In [11]:
bed.rooms.rooms["bedroom"][0].rooms.rooms["bedroom"]

[Object(bed, static, Rooms({'bedroom': [Object(bed, static, Rooms({...}))], 'officeroom': [], 'livingroom': []}))]

In [20]:
foo.location

'bedroom'

In [11]:
import random


class Object:
    def __init__(
        self,
        name: str,
        type: str,
        probs: dict,
        rooms: list = ["bedroom", "officeroom", "livingroom"],
    ) -> None:
        """Entity, e.g. human, object, room.

        Args
        ----
        name: e.g., Tae, laptop, bed
        type: e.g., static, independent, dependent
        probs:

            if type == "static", e.g., {"officeroom": 0, "livingroom": 0, "bedroom": 1}
            elif type == "independent", e.g., {"bedroom": 0.5, "livingroom": 0.5}
            elif type == "dependent", e.g., {"Tae": 0.7, "Michael": 0.1, "Vincent": 0.3}
            elif type == "agent", e.g., {"officeroom": 0.333, "livingroom": 0.333,
            "bedroom": 0.333}

        """
        self.name = name
        self.type = type
        assert self.type in ["static", "independent", "dependent", "agent"]
        self.probs = probs
        self.rooms = rooms
        self.move()

    def move(self) -> None:
        if self.type == "static":
            pass

        self.loc = random.choices(
            list(self.probs.keys()),
            weights=list(self.probs.values()),
            k=1,
        )[0]

    def __repr__(self) -> str:
        return f"Entity(name: {self.name}, type: {self.type}, location: {self.loc})"

    def change_location(self, loc: str) -> None:
        self.loc = loc


foo = Object("foo", "static", {"bedroom": 1.0, "livingroom": 0.2})
foo.loc, foo.dependent_probs


('bedroom', None)

In [77]:
import random
from pprint import pprint


class Pomdp:
    def __init__(
        self, rooms: list = ["bedroom", "officeroom", "livingroom"], seed: int = 42
    ) -> None:
        random.seed(seed)
        self.rooms = rooms

    def reset(self):
        self.objects = []

        # add static objects
        self.objects.append(
            Object(
                "bed",
                "static",
                {"officeroom": 0, "livingroom": 0, "bedroom": 1},
            )
        )
        self.objects.append(
            Object(
                "desk",
                "static",
                {"officeroom": 1, "livingroom": 0, "bedroom": 0},
            )
        )
        self.objects.append(
            Object(
                "table",
                "static",
                {"officeroom": 0, "livingroom": 1, "bedroom": 0},
            )
        )

        # add independent objects
        self.objects.append(
            Object(
                "Tae",
                "independent",
                {"officeroom": 0.5, "livingroom": 0.5, "bedroom": 0},
            )
        )
        self.objects.append(
            Object(
                "Michael",
                "independent",
                {"officeroom": 0, "livingroom": 0.5, "bedroom": 0.5},
            )
        )
        self.objects.append(
            Object(
                "Vincent",
                "independent",
                {"officeroom": 0.5, "livingroom": 0, "bedroom": 0.5},
            )
        )

        # add dependent objects
        self.objects.append(
            Object(
                "laptop",
                "dependent",
                {"officeroom": 0.5, "livingroom": 0.5, "bedroom": 0},
                {"Tae": 0.7, "Michael": 0.1, "Vincent": 0.3},
            )
        )
        self.objects.append(
            Object(
                "phone",
                "dependent",
                {"officeroom": 0, "livingroom": 0.5, "bedroom": 0.5},
                {"Tae": 0.3, "Michael": 0.1, "Vincent": 0.7},
            )
        )

        # add agent
        self.objects.append(
            Object(
                "agent",
                "agent",
                {"officeroom": 0.333, "livingroom": 0.333, "bedroom": 0.333},
            )
        )

        # get basic info
        self.num_rooms = len(self.rooms)

        self.num_static_objects = sum(
            [1 for obj in self.objects if obj.type == "static"]
        )
        self.num_independent_objects = sum(
            [1 for obj in self.objects if obj.type == "independent"]
        )
        self.num_dependent_objects = sum(
            [1 for obj in self.objects if obj.type == "dependent"]
        )

        self.num_hidden_states = self.num_rooms ** (
            self.num_independent_objects + self.num_dependent_objects + 1
        )
        self.num_observations = (
            2 ** (self.num_independent_objects + self.num_dependent_objects)
            * self.num_rooms
        )
        self.time = 0

        return self._get_observation()

    def _compute_hidden_state(self):
        if self.time == 0:
            self.hidden_state_room_perspective = {room: [] for room in self.rooms}
            for obj in self.objects:
                self.hidden_state_room_perspective[obj.loc].append(obj)

            self.hidden_state_kg_perspective = [
                (obj.name, "AtLocation", obj.loc)
                for room, objects in self.hidden_state_room_perspective.items()
                for obj in objects
            ]
        else:
            for room, objects in self.hidden_state_room_perspective.items():
                independent_objects = [
                    obj for obj in objects if obj.type == "independent"
                ]
                dependent_objects = [obj for obj in objects if obj.type == "dependent"]

                io_do_pairs = []
                for do in dependent_objects:
                    matches =[]
                    for io in independent_objects:
                        if io.name in list(do.dependent_probs.keys()):
                            matches.append(io)
                    if len(matches) > 0:
                        match = random.choice(matches)
                        if random.random() < do.dependent_probs[match.name]:
                            io_do_pairs.append((match, do))
                
                for io in independent_objects:
                    if random.random() < io.probs[room]:
                        


                
                for obj in objects:
                    if obj.type == "static":
                        continue

                    if obj.type == "independent":
                        obj.change_location(
                            random.choices(
                                list(obj.probs.keys()),
                                weights=list(obj.probs.values()),
                                k=1,
                            )[0]
                        )
                    elif obj.type == "dependent":
                        if (
                            random.random()
                            < obj.dependent_probs[
                                self.hidden_state_room_perspective[obj.loc][0].name
                            ]
                        ):
                            obj.change_location(
                                random.choices(
                                    list(obj.probs.keys()),
                                    weights=list(obj.probs.values()),
                                    k=1,
                                )[0]
                            )

    def _get_observation(self):
        self._compute_hidden_state()
        for obj in self.objects:
            if obj.name == "agent":
                self.agent_location = obj.loc

        self.sub_graph_room_perspective = [
            obj for obj in self.objects if obj.loc == self.agent_location
        ]
        self.sub_graph_kg_perspective = [
            (obj.name, "AtLocation", obj.loc, self.time)
            for obj in self.sub_graph_room_perspective
        ]
        self.question = random.choice(self.hidden_state_kg_perspective)

        return self.sub_graph_kg_perspective, self.question

    def step(self):
        self.time += 1
        self._get_observation()


pomdp = Pomdp(seed=42)
observation, question = pomdp.reset()
print(f"observation: {observation}")
print()
print(f"question: {question}")

# pprint(f"objects: {pomdp.objects}")
# print()
# pprint(f"hidden_state_room_perspective: {pomdp.hidden_state_room_perspective}")
# print()
# pprint(f"hidden_state_kg_perspective: {pomdp.hidden_state_kg_perspective}")
# print()
# pprint(f"sub_graph_room_perspective: {pomdp.sub_graph_room_perspective}")
# print()
# pprint(f"sub_graph_kg_perspective: {pomdp.sub_graph_kg_perspective}")
# print()
# pprint(f"question: {pomdp.question}\n")
# print()


observation: [('table', 'AtLocation', 'livingroom', 0), ('laptop', 'AtLocation', 'livingroom', 0), ('phone', 'AtLocation', 'livingroom', 0), ('agent', 'AtLocation', 'livingroom', 0)]

question: ('bed', 'AtLocation', 'bedroom')


In [1]:
import random

random.random()

0.4183346213273226

In [78]:
observation, question = pomdp.reset()
pomdp.hidden_state_room_perspective

{'bedroom': [Entity(name: bed, type: static, location: bedroom),
  Entity(name: Michael, type: independent, location: bedroom),
  Entity(name: Vincent, type: independent, location: bedroom)],
 'officeroom': [Entity(name: desk, type: static, location: officeroom),
  Entity(name: laptop, type: dependent, location: officeroom),
  Entity(name: agent, type: agent, location: officeroom)],
 'livingroom': [Entity(name: table, type: static, location: livingroom),
  Entity(name: Tae, type: independent, location: livingroom),
  Entity(name: phone, type: dependent, location: livingroom)]}

In [62]:
pomdp.objects[0].change_location("livingroom")

In [63]:
pomdp.objects[0]

Entity(name: bed, type: static, location: livingroom)

In [64]:
pomdp.objects[0].change_location("officeroom")

In [65]:
pomdp.objects[0]

Entity(name: bed, type: static, location: officeroom)

In [2]:
pomdp.time


0

In [50]:
pomdp.entities[0]


In [4]:
baz = {"foo": 0.2, "bar": 0.8}
random.sample(baz)


TypeError: sample() missing 1 required positional argument: 'k'

In [11]:
random.choice?

Signature: random.choice(seq)
Docstring: Choose a random element from a non-empty sequence.
File:      /usr/lib/python3.9/random.py
Type:      method

In [21]:
random.choices(list(baz.keys()), weights=list(baz.values()), k=1)[0]


'bar'

In [ ]:
import random


class Pomdp:
    def __init__(
        self,
        humans: dict,
        entities: dict,
        world_knowledge: list,
        init_probs: list,
        seed: int = 42,
    ) -> None:
        """POMDP"""
        self.seed = seed
        random.seed(self.seed)
        self.humans = humans
        self.objects = entities
        self.world_knowledge = world_knowledge

        self.static_objects = [
            entity
            for entity, characteristic in entities.items()
            if characteristic["type"] == "static_object"
        ]
        self.non_static_objects = [
            entity
            for entity, characteristic in entities.items()
            if characteristic["type"] == "non_static_object"
        ]
        self.rooms = [
            entity
            for entity, characteristic in entities.items()
            if characteristic["type"] == "room"
        ]

        self.objects = self.static_objects + self.non_static_objects
        self.world_knowledge = world_knowledge
        self.init_probs = init_probs

        self.num_humans = len(self.humans)  # number of humans
        self.num_static_objects = len(self.static_objects)  # number of static objects
        self.num_non_static_objects = len(
            self.non_static_objects
        )  # number of static objects
        self.num_rooms = len(self.rooms)  # number of rooms

        self.num_hidden_states = self.num_rooms ** (
            self.num_humans + self.num_non_static_objects + 1
        )
        self.num_observations = (
            2 ** (self.num_humans + self.num_non_static_objects) * self.num_rooms
        )
        self._init_environment()

    def _init_environment(self) -> None:
        """Populate the rooms with objects and humans at uniformly random."""
        self.env = {
            "bedroom": {"static_objects": []},
            "non_static_objects": [],
            "humans": [],
        }
        self.env["bedroom"] = {"static_object": ["bed"]}
        self.env["officeroom"] = {"static_object": ["desk"]}
        self.env["livingroom"] = {"static_object": ["table"]}

        for human in self.humans:
            room = random.choice(["bedroom", "officeroom", "livingroom"])
            self.env[room].append(human)

        for obj in self.non_static_objects:
            room = random.choice(["bedroom", "officeroom", "livingroom"])
            self.env[room].append(obj)

        room = random.choice(["bedroom", "officeroom", "livingroom"])
        self.env[room].append("agent")

    def step(self, action: int) -> tuple:
        """Take a step in the environment."""
        self.env["agent"] = ("AtLocation", self.rooms[action])


config = {
    "seed": 42,
    "humans": {
        "Tae": {
            "room": {"bedroom": 0.7, "officeroom": 0.1, "livingroom": 0.2},
            "object": {
                "laptop": 0.3,
                "phone": 0.3,
            },
        },
        "Michael": {
            "room": {"bedroom": 0.2, "officeroom": 0.7, "livingroom": 0.1},
            "object": {
                "laptop": 0.9,
            },
        },
        "Vincent": {
            "room": {"bedroom": 0.1, "officeroom": 0.2, "livingroom": 0.7},
            "object": {
                "phone": 0.8,
            },
        },
    },
    "entities": {
        "bedroom": {"type": "room"},
        "officeroom": {"type": "room"},
        "livingroom": {"type": "room"},
        "bed": {"type": "static_object"},
        "desk": {"type": "static_object"},
        "table": {"type": "static_object"},
        "laptop": {"type": "non_static_object"},
        "phone": {"type": "non_static_object"},
        "Tae": {"type": "human"},
        "Michael": {"type": "human"},
        "Vincent": {"type": "human"},
        "agent": {"type": "agent"},
    },
    "world_knowledge": [
        ("static_object", "PartOf", "room"),
        ("non_static_object", "AtLocation", "static_object"),
        ("human", "AtLocation", "room"),
        ("agent", "AtLocation", "room"),
        ("human", "Owns", "non_static_object"),
    ],
    "init_probs": [
        {"bed": [("PartOf", "bedroom", 1.0)]},
        {"desk": [("PartOf", "officeroom", 1.0)]},
        {"bed": [("PartOf", "livingroom", 1.0)]},
        {"laptop": [("AtLocation", "desk", 0.7), ("AtLocation", "bed", 0.3)]},
        {"phone": [("AtLocation", "table", 0.7), ("AtLocation", "bed", 0.3)]},
        {
            "Tae": [
                ("AtLocation", "bedroom", 0.7),
                ("AtLocation", "officeroom", 0.1),
                ("AtLocation", "livingroom", 0.2),
            ]
        },
        {
            "Michael": [
                ("AtLocation", "bedroom", 0.2),
                ("AtLocation", "officeroom", 0.7),
                ("AtLocation", "livingroom", 0.1),
            ]
        },
        {
            "Vincent": [
                ("AtLocation", "bedroom", 0.1),
                ("AtLocation", "officeroom", 0.2),
                ("AtLocation", "livingroom", 0.7),
            ]
        },
        {
            "agent": [
                ("AtLocation", "bedroom", 0.333),
                ("AtLocation", "officeroom", 0.333),
                ("AtLocation", "livingroom", 0.333),
            ]
        },
    ],
}

pomdp = Pomdp(**config)
pomdp.humans, pomdp.objects, pomdp.rooms


In [ ]:
class Entity:
    def __init__(self, name: str, kind: str) -> None:
        """Entity"""
        self.name = name
        self.kind


In [ ]:
pomdp.env


In [ ]:
pomdp.step(1)


In [ ]:
pomdp.env


In [ ]:
random.choice(["bedroom", "officeroom", "livingroom"])


In [ ]:
pomdp.humans.keys(), pomdp.rooms


In [ ]:
pomdp.hidden_states


In [ ]:
pomdp.num_humans, pomdp.num_static_objects, pomdp.num_non_static_objects, pomdp.num_rooms, pomdp.num_hidden_states, pomdp.num_observations


In [ ]:
pomdp.num_observations, pomdp.num_hidden_states


In [ ]:
list(config["objects"].keys())


In [ ]:
len(config["objects"])


In [ ]:
w = np.array(
    [
        [0, 0.4, 0.8, 0],
        [1, 0, 0, 0.7],
        [0, 0, 0, 0.3],
        [0, 0.6, 0.2, 0],
    ]
)
w_ = np.eye(4)

foo = []
for i in range(1000):
    w_ = w @ w_
    print(w_)


In [ ]:
import numpy as np

alpha = 0.3
beta = 0.7

w = np.array(
    [
        [1 - beta, beta, 0, alpha * beta],
        [(1 - alpha) * beta, 1 - beta, 0, 0],
        [0, 0, 1 - beta, (1 - alpha) * beta],
        [alpha * beta, 0, beta, 1 - beta],
    ]
)
w_ = np.eye(4)

foo = []
for i in range(1000):
    w_ = w @ w_
    print(w_)


In [ ]:
import numpy as np

alpha = 0.3
beta = 0.7

w = np.array(
    [
        [0.3, 0.7, 0, 0],
        [0.2, 0.3, 0, 0],
        [0, 0, 0.3, 0.5],
        [0.5, 0, 0.7, 0.5],
    ]
)
w_ = np.eye(4)

foo = []
for i in range(1000):
    w_ = w @ w_
    print(w_)


In [ ]:
w = np.array(
    [
        [0, 0, 1],
        [1, 0, 0],
        [0, 1, 0],
    ]
)
w_ = np.eye(3)

foo = []
for i in range(1000):
    w_ = w @ w_
    print(w_)


In [ ]:
import numpy as np

alpha = 0.3
beta = 0.7

w = np.array(
    [
        [1 - alpha, alpha * (1 - beta), 0, alpha * beta],
        [alpha * (1 - beta), 1 - alpha, 0, 0],
        [0, 0, 1 - alpha, alpha * (1 - beta)],
        [alpha * beta, 0, alpha * (1 - beta), 1 - alpha],
    ]
)
w_ = np.eye(4)

foo = []
for i in range(1000):
    w_ = w @ w_
    print(w_)
    foo.append(w_[0, 0])


In [ ]:
np.random.randint(0, 10, 5)


In [ ]:
import numpy as np


w = np.array(
    [
        [0, 0.2, 0],
        [1, 0, 1],
        [0, 0.8, 0],
    ]
)
x = np.array([1 / 3, 1 / 3, 1 / 3])

for i in range(100):
    x = w @ x
    print(x)
    print()


In [ ]:
np.random.randint(0, 10, 5)


In [ ]:
import numpy as np

w = np.random.randint(0, 10, (5, 5))
w = w / w.sum(axis=1)[:, np.newaxis]

x = np.random.randint(0, 10, 5)
x = x / x.sum()
# foo, foo.sum(axis=1)

for i in range(100):
    x = w @ x
    print(x)


In [ ]:
x


In [ ]:
w


In [ ]:
foo.sum(axis=1)[:, np.newaxis]


In [ ]:
import numpy as np


w = np.array(
    [
        [3 / 4, 1 / 4, 0, 0, 0],
        [3 / 4, 0, 1 / 4, 0, 0],
        [3 / 4, 0, 0, 1 / 4, 0],
        [3 / 4, 0, 0, 0, 1 / 4],
        [1, 0, 0, 0, 0],
    ]
)
w_ = np.eye(5)

for i in range(100):
    w_ = np.dot(w, w_)
    print(w_)


In [ ]:
w.shape, x.shape
